# DSU Worker (Kaggle)

Run all: sets up GPU wheels and launches the watcher. Uses Kaggle's /kaggle/working as the DSU root.


In [ ]:
# One-click venv + CUDA-correct installs (Kaggle)
import os, sys, subprocess, pathlib, re
BASE = pathlib.Path('/kaggle/working/M4L-Demucs')
VENV = BASE / '.venv'
PY = VENV / 'bin' / 'python'
BASE.mkdir(parents=True, exist_ok=True)

print('Kaggle bootstrap. BASE=', BASE)
print(subprocess.getoutput('nvidia-smi'))

# Create isolated venv (once)
if not VENV.exists():
    subprocess.check_call([sys.executable, '-m', 'venv', str(VENV)])
    subprocess.check_call([str(PY), '-m', 'pip', 'install', '--upgrade', 'pip', 'setuptools', 'wheel'])

# Detect CUDA and choose torch index/spec
smi = subprocess.getoutput('nvidia-smi')
m = re.search(r'CUDA Version:\s*([0-9.]+)', smi)
if not m:
    raise RuntimeError('GPU not detected. In Kaggle: Settings → Accelerator → GPU, then Run all.')
ver = m.group(1)
if ver.startswith('12.6'):
    fl, torch_ver, ta_ver, idx = ('cu126','2.8.0','2.8.0','https://download.pytorch.org/whl/cu126')
elif ver.startswith('12.4'):
    fl, torch_ver, ta_ver, idx = ('cu124','2.5.1','2.5.1','https://download.pytorch.org/whl/cu124')
elif ver.startswith('12.'):
    fl, torch_ver, ta_ver, idx = ('cu126','2.8.0','2.8.0','https://download.pytorch.org/whl/cu126')
elif ver.startswith('11.8'):
    fl, torch_ver, ta_ver, idx = ('cu118','2.1.2','2.1.2','https://download.pytorch.org/whl/cu118')
else:
    fl, torch_ver, ta_ver, idx = ('cu126','2.8.0','2.8.0','https://download.pytorch.org/whl/cu126')

print('CUDA flavor:', fl)

# Torch/torchaudio strictly from CUDA index inside venv
subprocess.check_call([str(PY), '-m', 'pip', 'install', '--extra-index-url', idx,
                       f'torch=={torch_ver}+{fl}', f'torchaudio=={ta_ver}+{fl}'])

# Demucs and deps inside venv (no global bleed)
subprocess.check_call([str(PY), '-m', 'pip', 'install', 'demucs==4.0.1', 'torchcodec', 'ffmpeg-python', 'numpy==2.3.4'])

# Verify
subprocess.run([str(PY), '-c', "import torch, demucs; print('Boot OK.', 'torch', torch.__version__, 'cuda', torch.cuda.is_available(), 'demucs', demucs.__version__)"] , check=True)


In [ ]:
# Launch watcher (Kaggle)
import urllib.request, pathlib, os
BASE = pathlib.Path(os.environ.get('DSU_ROOT','/kaggle/working/M4L-Demucs'))
w = BASE/'colab_watcher.py'
urllib.request.urlretrieve('https://raw.githubusercontent.com/VSTOPIA/Doctor-Sample-Unit-DSU/main/colab_watcher.py', w)
print('Watcher saved to', w)
!python -u - <<'PY'
import os
os.environ['DSU_ROOT'] = '/kaggle/working/M4L-Demucs'
os.environ['DSU_REMOTE_JOBS_URL'] = 'https://raw.githubusercontent.com/VSTOPIA/Doctor-Sample-Unit-DSU/main/remote_jobs.jsonl'
print('DSU_ROOT=', os.environ['DSU_ROOT'])
print('DSU_REMOTE_JOBS_URL=', os.environ['DSU_REMOTE_JOBS_URL'])
PY

# Run watcher with venv python (from cell 1)
import subprocess
subprocess.call([str(PY), str(w)])
